In [ ]:
import keras
import tensorflow as tf
import keras.backend as K

from keras.layers import Conv2D, SeparableConv2D, Conv2DTranspose, add, ReLU, Dropout, Reshape, Permute
from keras.layers import Activation, Dense, Add
from keras.models import Model

In [ ]:
Add()

In [4]:
from models import matting_net

model = matting_net((256,256,4), android=True)

# model.load_weights("./trained_models/portrait_seg_matting_256_191017_val_loss_0.0404_val_acc_0.9725_focal_1312.8737.h5")
model.load_weights("./trained_models/initial.h5")

In [5]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 4)  0                                            
__________________________________________________________________________________________________
lambda_21 (Lambda)              (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 256, 256, 8)  224         lambda_21[0][0]                  
__________________________________________________________________________________________________
re_lu_47 (ReLU)                 (None, 256, 256, 8)  0           conv2d_13[0][0]                  
____________________________________________________________________________________________

In [4]:
# converting

In [10]:
outPath = "./tflite/initial_new_softmax.tflite"

input_names = [node.op.name for node in model.inputs]
output_names = [node.op.name for node in model.outputs]
print(input_names)
print(output_names)

sess = K.get_session()
converter = tf.lite.TFLiteConverter.from_session(sess, model.inputs, model.outputs)

tflite_model = converter.convert()
open(outPath, "wb").write(tflite_model)

['input_3']
['output_2/Sigmoid']
INFO:tensorflow:Froze 166 variables.
INFO:tensorflow:Converted 166 variables to const ops.


3743300

In [11]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 4)  0                                            
__________________________________________________________________________________________________
lambda_21 (Lambda)              (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 256, 256, 8)  224         lambda_21[0][0]                  
__________________________________________________________________________________________________
re_lu_47 (ReLU)                 (None, 256, 256, 8)  0           conv2d_13[0][0]                  
____________________________________________________________________________________________

In [8]:
import numpy as np 
import matplotlib.pyplot as plt
random_img = np.random.rand(1,256,256,4)
plt.imshow(model.predict(random_img).reshape(256, 256))

In [9]:
model.predict(random_img)

array([[[[0.08880895],
         [0.05536437],
         [0.04643607],
         ...,
         [0.16278192],
         [0.15298858],
         [0.17478547]],

        [[0.02744746],
         [0.01178122],
         [0.00793764],
         ...,
         [0.11677435],
         [0.12056005],
         [0.15188423]],

        [[0.00792524],
         [0.0020645 ],
         [0.00114462],
         ...,
         [0.04998979],
         [0.06365827],
         [0.09672934]],

        ...,

        [[0.22294816],
         [0.26938742],
         [0.24629837],
         ...,
         [0.99343944],
         [0.9796947 ],
         [0.8617343 ]],

        [[0.21906966],
         [0.2925772 ],
         [0.27673852],
         ...,
         [0.98983043],
         [0.9735173 ],
         [0.85998684]],

        [[0.21252081],
         [0.24161884],
         [0.23612165],
         ...,
         [0.94108415],
         [0.90316904],
         [0.72567236]]]], dtype=float32)

In [12]:
import numpy as np 

img = np.random.rand(124,124, 3)

In [15]:
import cv2

dim = (256, 192)

cv2.resize(img, dim[::-1])

array([[[0.65101339, 0.74761895, 0.9025449 ],
        [0.78758948, 0.47273871, 0.74891978],
        [0.88495639, 0.18710234, 0.56026201],
        ...,
        [0.54611734, 0.73812462, 0.82363478],
        [0.38238132, 0.79103271, 0.55930895],
        [0.23471498, 0.76903251, 0.30371817]],

       [[0.70217292, 0.77486737, 0.78193571],
        [0.80376633, 0.55469629, 0.65609756],
        [0.87412101, 0.32394544, 0.51102347],
        ...,
        [0.4697521 , 0.75456138, 0.83916124],
        [0.42360617, 0.77038108, 0.60417612],
        [0.3862941 , 0.72597392, 0.36168973]],

       [[0.81154848, 0.8331226 , 0.52408159],
        [0.83835133, 0.72991594, 0.45765006],
        [0.85095573, 0.61650655, 0.40575486],
        ...,
        [0.30648849, 0.78970202, 0.87235572],
        [0.51174206, 0.72622933, 0.70009902],
        [0.71035978, 0.63391761, 0.48562894]],

       ...,

       [[0.74893844, 0.70846688, 0.445423  ],
        [0.66449035, 0.52910011, 0.57090228],
        [0.56636189, 0

In [10]:
np.expand_dims(arr, axis=-1).shape

(124, 124, 1)

In [ ]:
cv2.imread(cv2.IMREAD_GRAY)